Dans ce script, nous allons tester différents modèles de type convolution Nelle pour examiner leur performance. Nous allons ensuite tester s'ils sont robustes lorsqu'on applique une rotation à l'image d'entrée.

In [ ]:
import retinoto_py as fovea
args = fovea.Params(do_mask=True, n_train_stop=200_000, n_val_stop=10_000, in_memory=True)
args = fovea.Params(do_mask=True, n_train_stop=0, n_val_stop=0, batch_size=64, in_memory=False)
args

# transfer learning of each network on the  masked dataset

In [6]:
%ls -ltr cached_data/20*

zsh:1: no matches found: cached_data/20*


In [ ]:
# %rm "cached_data/20_model_name=convnext_base_dataset=bbox.lock"   # FORCING RECOMPUTE
%rm "cached_data/20_*.lock"   # FORCING RECOMPUTE
# %rm "cached_data/20_model_name=resnet101_dataset=full.lock"   # FORCING RECOMPUTE


In [ ]:
model_name = args.model_name
for dataset in ['bbox']: # fovea.all_datasets:
    name = f'20_model_name={model_name}_dataset={dataset}'
    model_filename, json_filename = fovea.do_learning(args, dataset, name)
    print(50*'.')

Training model convnext_base, file= cached_data/20_model_name=convnext_base_dataset=bbox.pth - image_size=224
Starting learning...


Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch=1/20:   0%|          | 0/9614 [00:00<?, ?it/s]

## Model comparison

TODO Faire un nouveau notebook 23_

In [ ]:
model_name = args.model_name
for dataset in fovea.all_datasets:
    results = fovea.pd.read_json(args.data_cache / f'20_model_name={model_name}_dataset={dataset}.json')
    print(model_name, dataset, results.tail(1)['acc_val'])

In [ ]:
results_13 = fovea.pd.read_json(args.data_cache / '13_model_comparison.json')
results_13

## Plot learning evolution

In [ ]:
fig, ax = fovea.plt.subplots()
for dataset in fovea.all_datasets:
    color = 'r'
    lw = 1

    name = f'20_model_name={model_name}_dataset={dataset}'
    model_filename, json_filename = fovea.do_learning(args, dataset, name)

    df_train = fovea.pd.read_json(json_filename, orient='records')

    df_train_roll = df_train.rolling(window=5, min_periods=1, center=False).mean()
    ax = df_train_roll.plot(x='total_image', y='acc_train', 
                        c=color, ls='dashed', lw=lw,
                        grid=True, ax=ax, label='TRAIN: ' + args.model_name)    
    ax = df_train_roll.plot(x='total_image', y='acc_val', 
                        c=color, lw=lw,
                        grid=True, ax=ax, label='VAL: ' + args.model_name)   

In [ ]:
df_train